<a href="https://colab.research.google.com/github/guptasamz/CS_6376_Hybrid_Embedded_Sys/blob/main/4_join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#to solve this go and read the class lecture on map reduce where we discussed a trivial join example.
#here the join key is playerid and year id

In [1]:
#install mrjob
!pip install mrjob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 KB 5.6 MB/s eta 0:00:00


In [2]:
from google.colab import files

# upload dataset
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Batting.csv to Batting.csv
Saving Salaries.csv to Salaries.csv
User uploaded file "Batting.csv" with length 6488747 bytes
User uploaded file "Salaries.csv" with length 700024 bytes


In [4]:
from google.colab import files

# upload mrjob.conf under current directory(/content/mrjob.conf)
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [156]:
# from traitlets.config import keyvalue_description
%%file 4_join.py
#write the magic string to save it as 4_join.py
from mrjob.job import MRJob
from mrjob.step import MRStep

'''
Join the batting and salaries data for Barry Bonds per year.

The output should be the combined CSV string of batting and salaries data (one per year).

Use 'join' as the key for the final output in the reducer.

E.g:
"join"  "bondsba01,1986,1,PIT,NL,113,413,72,92,26,3,16,48,36,7,65,102,2,2,2,2,41986,PIT,NL,bondsba01,60000"

Schema:
Salaries: yearID	teamID	lgID	playerID	salary
Batting: playerID	yearID	stint	teamID	lgID	G	AB	R	H	2B	3B	HR	RBI	SB	CS	BB	SO

Hints: 
Use split to split the CSV lines (e.g. s = line.split(','))
Both files are sent to the mapper. Use the length of the lines to determine which is which.
'''


class Join(MRJob):
    # Using SORT_VALUES to sort on all fields, not just first. Read about sort values in https://buildmedia.readthedocs.org/media/pdf/mrjob/latest/mrjob.pdf
    MRJob.SORT_VALUES = False
    #PARTITIONER = True
 

    def mapper(self, _, line):
        lines = line.split(',')
        lines_len = len(lines)

          # playerID	yearID	stint	teamID	lgID	G	AB	R	H	2B	3B	HR	RBI	SB	CS	BB	SO
        if (lines_len == 24) & (lines[0] == "bondsba01"):
          indexes = [ 0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
          lines = [lines[x] for x in indexes]
          key = int(lines[1])
          value = ','.join(lines)
          yield key, value
        elif (lines_len == 5) & (lines[3] == "bondsba01"): 
          key = int(lines[0])
          value = ','.join(lines)
          yield key, value

        
    def reducer(self, year, lines):
        checker = [x for x in lines]
        if len(checker) > 1:
          #Drop the columns that are not present in the schema 
          #Break the list and merge it all into one thing  - sum(checker, [])
          yield "join", ','.join(checker)
        else:
          pass  
        
    def reduce_sort(self, _, values):
        for value in sorted(values):
            yield "join", value
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper, reducer=self.reducer),
            MRStep(reducer=self.reduce_sort)
        ]


if __name__ == '__main__':
    Join.run()

Overwriting 4_join.py


## Step 1. Run MR program locally(Colab)

In [157]:
#write the command to run it first to check for error
%%time
!python 4_join.py Batting.csv Salaries.csv > 4_join.out

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/4_join.root.20230324.234753.730525
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/4_join.root.20230324.234753.730525/output
Streaming final output from /tmp/4_join.root.20230324.234753.730525/output...
Removing temp directory /tmp/4_join.root.20230324.234753.730525...
CPU times: user 12.1 ms, sys: 11.2 ms, total: 23.3 ms
Wall time: 626 ms


In [158]:
!cat 4_join.out

"join"	"bondsba01,1986,1,PIT,NL,113,413,72,92,26,3,16,48,36,7,65,102,2,2,2,2,4,1986,PIT,NL,bondsba01,60000"
"join"	"bondsba01,1987,1,PIT,NL,150,551,99,144,34,9,25,59,32,10,54,88,3,3,0,3,4,1987,PIT,NL,bondsba01,100000"
"join"	"bondsba01,1988,1,PIT,NL,144,538,97,152,30,5,24,58,17,11,72,82,14,2,0,2,3,1988,PIT,NL,bondsba01,220000"
"join"	"bondsba01,1989,1,PIT,NL,159,580,96,144,34,6,19,58,32,10,93,93,22,1,1,4,9,1989,PIT,NL,bondsba01,360000"
"join"	"bondsba01,1990,1,PIT,NL,151,519,104,156,32,3,33,114,52,13,93,83,15,3,0,6,8,1990,PIT,NL,bondsba01,850000"
"join"	"bondsba01,1991,1,PIT,NL,153,510,95,149,28,5,25,116,43,13,107,73,25,4,0,13,8,1991,PIT,NL,bondsba01,2300000"
"join"	"bondsba01,1992,1,PIT,NL,140,473,109,147,36,5,34,103,39,8,127,69,32,5,0,7,9,1992,PIT,NL,bondsba01,4800000"
"join"	"bondsba01,1993,1,SFN,NL,159,539,129,181,38,4,46,123,29,12,126,79,43,2,0,7,11,1993,SFN,NL,bondsba01,4516666"
"join"	"bondsba01,1994,1,SFN,NL,112,391,89,122,18,1,37,81,29,9,74,43,18,6,0,3,3,1994,SFN,NL,bondsba01,

In [91]:
# code
l = ['hello', 'geek', 'have',
   'a', '1', 'day']
 
# this will join all the
# elements of the list with ' '
l = ','.join(l)
print(l)

hello,geek,have,a,1,day


## Step 2. Run MR program on AWS EMR

### Please copy  Batting.csv and Salaries.csv into your S3 bucket.

In [ ]:
#run and save output as 4_join.out
#note that you can pass two files -- in any order to the mrjob
#you might first check for errors before cascading the output it to 4_join.out
%%time
!python 4_join.py -r emr s3://<s3 url of the batting dataset> s3://<s3 url of the salaries dataset> --cloud-tmp-dir=s3://<s3 url of tmp directory> --cluster-id=<cluster ID> --conf-path <mrjob.conf file path in colab> > 4_join.out

In [ ]:
!cat 4_join.out

In [161]:
def test_count():
    f = open('4_join.out')
    lines = f.readlines()
    f.close()
    
    found_line = False
    for  line in lines:
        if line[:-1] in [
            '"join"\t"bondsba01,1986,1,PIT,NL,113,413,72,92,26,3,16,48,36,7,65,102,2,2,2,2,41986,PIT,NL,bondsba01,60000"',
            '"join"\t"bondsba01,1986,1,PIT,NL,113,413,72,92,26,3,16,48,36,7,65,102,2,2,2,2,4,1986,PIT,NL,bondsba01,60000"']:
            found_line = True
    if not found_line:
        assert False

    
    assert len(lines) == 22

test_count()